In [1]:
import zipfile
import os
from datasets import load_from_disk

# Загружаем тренировочный датасет из локального диска
train_dataset = load_from_disk('./train_dataset')
val_dataset = load_from_disk('./val_dataset')


/Users/evgeny/Desktop/study/bot_telegram_request_message/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!huggingface-cli login
#

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset

model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    # load_in_4bit=True,
    low_cpu_mem_usage= True,
    offload_folder="offload_weights",
)

KeyboardInterrupt: 

In [7]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

tokenizer.add_special_tokens({'additional_special_tokens': ['<context>', '<response>']})
model.resize_token_embeddings(len(tokenizer))

for param in model.parameters():
    param.requires_grad=False

model.gradient_checkpointing_enable() ### CHECK
model.enable_input_require_grads()  ###CHECK

In [8]:
from transformers import DataCollatorForLanguageModeling
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm import tqdm

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj"],
    inference_mode=False
)

model = get_peft_model(model, peft_config)

In [9]:
train_dataset = train_dataset

def tokenize_function(examples):
    result = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=120)
    return result

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/180000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [10]:
print("Trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))
print("Total parameters (excluding quantization):", sum(p.numel() for p in model.parameters()))


Trainable parameters: 2097152
Total parameters (excluding quantization): 3760476160


In [ ]:
#llama 7b 65536
2097152 / 65536 / 3276800 / 2097152 / 1048576

32.0

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/bot_telegram/results', # TODO: поменять + чекпоинты + квантизациz
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=1,
    evaluation_strategy="steps",
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/bot_telegram/logs',
    learning_rate=1e-4,
    fp16=True,
    gradient_accumulation_steps=8,
    save_steps=100,
    save_total_limit=3,
    optim="adamw_torch_fused",
    gradient_checkpointing=True,
)

optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

model.to('cuda')

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_val_dataset,
    # train_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    optimizers=(optimizer, None),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or train

Step,Training Loss,Validation Loss
